Some imports

In [1]:
%pylab inline
from ipywidgets import *
import scipy.linalg as sl
from scipy.special import jn
import scipy.sparse as ss
import h5py
import tqdm

Populating the interactive namespace from numpy and matplotlib


### Chebyshev time propagation

A thesis about the method: http://lup.lub.lu.se/luur/download?func=downloadFile&recordOId=3363166&fileOId=3363176



In [2]:
def timeprop_cheb(psi,H,dt,num_cheb=20):
    '''
    Propagate state psi acording to H for a time dt.
    Uses Chebyshev scheme. Appropriate for generic systems
    '''    
    cv=2*jn(arange(num_cheb+1),dt)
    cv[0]=cv[0]/2
    psi_m2=psi
    psi_m1=-1.0j*H*psi
    w=cv[0]*psi_m2+cv[1]*psi_m1
    for i in arange(2,num_cheb):
        psi_i=-2.0j*H*psi_m1+psi_m2
        w=w+cv[i]*psi_i
        psi_m2=psi_m1
        psi_m1=psi_i
    return w

Hamiltonian generation

In [3]:
dim=50
idm=ss.eye(dim);
odm=ss.diags(-ones(dim-1),1)+ss.diags(-ones(dim-1),-1);
H=ss.kron(idm,ss.kron(idm,odm)+ss.kron(odm,idm))+ss.kron(odm,ss.diags(-ones(dim*dim),0))

Initial wavefunction generation as a complex gaussian. 

In [4]:
%time
x,y,z=mgrid[0:dim,0:dim,0:dim]

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 13.4 µs


In [5]:
sigma=10
psi0=exp((-(x-dim/2)**2-(y-dim/2)**2-(z-dim/2)**2)/sigma**2)*exp(1j*x*0.2)
h5f = h5py.File('psi_t0.h5', 'w')
h5f.create_dataset('psi', data=abs(psi0)**2)
h5f.close()

psi0=psi0.reshape((dim**3,1))
psi0=psi0/norm(psi0)

Run time evolution and dump results in hdf5 files

In [6]:
dt=0.5
for ti in tqdm.tqdm_notebook(range(300)):
# or without tqdm
#for ti in range(300):
    psi=timeprop_cheb(psi0,H,dt,num_cheb=40)
    h5f = h5py.File('psi_t_'+str(ti).zfill(5)+'.h5', 'w')
    h5f.create_dataset('psi', data=abs(psi.reshape((dim,dim,dim)))**2)
    h5f.close()
    psi0=psi

HBox(children=(IntProgress(value=0, max=300), HTML(value='')))

KeyboardInterrupt: 

In [7]:
#!rm psi_t*

In [7]:
# dt=0.5
# for ti in range(300):
#     psi=timeprop_cheb(psi0,H,dt,num_cheb=40)
#     h5f = h5py.File('psi_t_'+str(ti).zfill(5)+'.h5', 'w')
#     h5f.create_dataset('psi', data=abs(psi.reshape((dim,dim,dim)))**2)
#     h5f.close()
#     psi0=psi